# Kim Week 4 - Assignment 2

Total Cost of Transportation =  22350.0

- Route_(1,_2) = 120.0
- Route_(1,_4) = 80.0
- Route_(2,_3) = 20.0
- Route_(3,_5) = 0.0
- Route_(5,_3) = 40.0
- Route_(5,_4) = 0.0
- Route_(5,_6) = 0.0
- Route_(6,_5) = 0.0
- Route_(7,_4) = 0.0
- Route_(7,_5) = 210.0
- Route_(7,_6) = 70.0


From the sensitivity report (the dual problem):

Total Cost of Transportation =  22350.0


In [4]:
import pandas as pd
from pulp import LpProblem, splitDict, LpVariable,LpContinuous,lpSum,GLPK,value, LpMinimize, GUROBI, LpStatus

nodes = [1,2,3,4,5,6,7]

node_data = {
    1:[200,0],
    2:[0,100],
    3:[0,60],
    4:[0,80],
    5:[0,170],
    6:[0,70],
    7:[300,0],
}

# make sure you have 11 arcs
arcs = {
    (1,2), (1,4), 
    (2,3), 
    (3,5), 
    (5,3), (5,4), (5,6),
    (6,5),
    (7,4), (7,5), (7,6) 
}

arc_data = {
    (1,2): [30,0,200], 
    (1,4): [40,0,200], 
    (2,3): [50,0,200], 
    (3,5): [35,0,200], 
    (5,3): [40,0,200], 
    (5,4): [30,0,200], 
    (5,6): [35,0,200],
    (6,5): [25,0,200],
    (7,4): [50,0,300], 
    (7,5): [45,0,300], 
    (7,6): [50,0,300], 
}

(supply, demand) = splitDict(node_data)
(costs, mins, maxs) = splitDict(arc_data)

vars = LpVariable.dicts("Route",arcs,None,None,LpContinuous)

for a in arcs:
    vars[a].bounds(mins[a], maxs[a])


prob = LpProblem("problem",LpMinimize)

prob += lpSum([vars[a]* costs[a] for a in arcs])

for n in nodes:
    prob += (supply[n]+ lpSum([vars[(i,j)] for (i,j) in arcs if j == n]) >=
             demand[n]+ lpSum([vars[(i,j)] for (i,j) in arcs if i == n])), \
            "Constraint %s"%n
    

prob.writeLP("discussion4v2.lp")

prob.solve(GUROBI(mip=False))

# un-comment for first run to get GLPK sen report
#prob4.solve(GLPK(options=['--ranges discussion4v2.sen']))


print("Total Cost of Transportation = ", value(prob.objective))

for v in prob.variables():
    print(v.name, "=", v.varValue)

print ("Status:", LpStatus[prob.status])

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 9 3900X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 7 rows, 11 columns and 22 nonzeros
Model fingerprint: 0xe9c20c25
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+01]
  Bounds range     [2e+02, 3e+02]
  RHS range        [6e+01, 3e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 6 rows, 10 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+03   4.750000e+01   0.000000e+00      0s
       6    2.2350000e+04   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.235000000e+04
Gurobi status= 2
Total Cost of Transportation =  22350.0
Route_(1,_2) = 120.0
Route_(1,_4) = 80.0
Route_(2,_3) = 20.0
Route_(3,_5) = 0.0
Route_(5,

In [ ]:

o = [{'name':name,'value':c.value(),'shadow price':c.pi,'slack': c.slack, 'lowerbound':c.getLb(), 'upperbound':c.getUb()} for name, c in prob.constraints.items()]

print(pd.DataFrame(o))

           name  value  shadow price  slack  lowerbound upperbound
0  Constraint_1    0.0           5.0    0.0        -200       None
1  Constraint_2    0.0          35.0    0.0         100       None
2  Constraint_3    0.0          85.0    0.0          60       None
3  Constraint_4    0.0          45.0    0.0          80       None
4  Constraint_5    0.0          45.0    0.0         170       None
5  Constraint_6    0.0          50.0    0.0          70       None
6  Constraint_7   20.0           0.0  -20.0        -300       None
